<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>
<h3>Eshan Ratnayake</h3>

<h2>Part 1: Scraping Data and Putting in Dataframe</h2>

In [1]:
import requests

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium 

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   3.07 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  15.97 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  18.51 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  23.40 MB/s


In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_random_wikipedia_page = requests.get(wikipedia_link)
page = raw_random_wikipedia_page.text

start = page.find('<table class="wikitable sortable">')
#finish = page.find('<table class="multicol" role="presentation" style="border-collapse: collapse; padding: 0; border: 0; background:transparent; width:100%;">')
finish = page.find('</table>')
table = page[start+34:finish]
#print(table)

In [3]:
colList = ["PostalCode", "Borough", "Neighbourhood"]
rowList = []

bs=BeautifulSoup(table, "lxml")
table_body=bs.find('tbody')
rows = table_body.find_all('tr')
count = 0
for row in rows:
    #count
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    rowList.append(cols)

df_table = pd.DataFrame(rowList, columns = colList)
#df_table.head(15)

In [4]:
df_table.dropna(inplace=True) 
df_table.drop(df_table[df_table.Borough == "Not assigned"].index, axis=0, inplace=True)
#df_table.head(15)

In [5]:
df_table = df_table.reset_index(drop=True)
#getting list of indices where Neighbourhoods are "Not assigned"
indexesWithNoNeighbouhood = df_table[df_table.Neighbourhood == "Not assigned"].index

#replacing all 'Not Assigned' Neghbourhoods with their Borough Name
for indexesWithNoNeighbouhood in indexesWithNoNeighbouhood:
    df_table.replace(df_table.iat[indexesWithNoNeighbouhood,2] ,df_table.iat[indexesWithNoNeighbouhood,1], inplace = True)

#df_table.drop_duplicates()
#df_table.head(100)

In [6]:
df_table=df_table.groupby(['PostalCode']).agg({'Borough': 'first','Neighbourhood':', '.join})
df_table.reset_index(inplace=True)
#df_table.columns = ["PostalCode", "Borough", "Neighbourhood"]
df_table = df_table[colList]
df_table.head()
#df_table.shape

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
#rowBehind = ["Placeholder", "Placeholder", "Placeholder"]
#count=0
#chain =0
#NeighbourhoodString = ""
#x=0

#for row in df_table.iterrows():
    #if they have the same PostalCode and Borough
#    if (row[1][0] == rowBehind[1][0] and row[1][1] == rowBehind[1][1]):  
#        chain = chain+1
#        if (chain > 1):
#            NeighbourhoodString = NeighbourhoodString + ", " + df_table.iat[count,2]
#        elif (chain > 0):
#            NeighbourhoodString = df_table.iat[count-chain,2] + ", " + df_table.iat[count,2]
#    else:
#        if (NeighbourhoodString != ""):
#            for x in range (0,chain+1):
#                df_table.iat[count-x-1,2] = NeighbourhoodString
#            #df_table.iat[count-chain-1,2] = NeighbourhoodString   
#        NeighbourhoodString = ""
#        chain = 0
        
#   rowBehind = row
#   count = count+1


In [8]:
df_table.head(103)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
df_table.drop_duplicates(inplace =True)
df_table = df_table.reset_index(drop=True)
#df_table.head(107)

In [10]:
df_table.shape

(103, 3)

<h2>Part 2: Add Latitude and Longitude Columns</h2>

In [33]:
df_LatLon = pd.read_csv("http://cocl.us/Geospatial_data") 
df_LatLon.columns = ["PostalCode", "Latitude", "Longitude"]
df_LatLon.head()

<class 'pandas.core.series.Series'>


In [34]:
df_combine = pd.merge(df_table, df_LatLon, on="PostalCode")
df_combine.head()

<class 'pandas.core.series.Series'>


<h2>Part 3: Clustering Neighbourhoods</h2>

In [ ]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

In [40]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_combine['Latitude'], df_combine['Longitude'], 
                                            df_combine['Borough'], df_combine['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [41]:
#Foursquare credentials
CLIENT_ID = 'YSK4NSSJEHR20EHPGGR22MO52KXJMOSS3PQSPB1ASHISUTRA' # your Foursquare ID
CLIENT_SECRET = 'YKSPEBXNZUPLG0WEIVQPNTCJLJBWKR4HNSAHM5BR22VTO1ND' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Your credentails:
CLIENT_ID: YSK4NSSJEHR20EHPGGR22MO52KXJMOSS3PQSPB1ASHISUTRA
CLIENT_SECRET:YKSPEBXNZUPLG0WEIVQPNTCJLJBWKR4HNSAHM5BR22VTO1ND


In [59]:
#filtering out for Boroughs with 'Toronto' in name
toronto_data = df_combine[df_combine['Borough'].str.contains('|'.join(['Toronto']))]

In [60]:
toronto_data

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [64]:
neighborhood_latitude = toronto_data.iat[0, 3] # neighborhood latitude value
neighborhood_longitude = toronto_data.iat[0, 4] # neighborhood longitude value
neighborhood_name = toronto_data.iat[0, 2] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [66]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=YSK4NSSJEHR20EHPGGR22MO52KXJMOSS3PQSPB1ASHISUTRA&client_secret=YKSPEBXNZUPLG0WEIVQPNTCJLJBWKR4HNSAHM5BR22VTO1ND&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [67]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bf78861351e3d163d9b3411'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b8daea1f964a520480833e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d11b941735',
         'name': 'Pub',
         'pluralName': 'Pubs',
         'primary': True,
         'shortName': 'Pub'}],
       'id': '4b8daea1f964a520480833e3',
       'location': {'address': '676 Kingston Rd.',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Main St.',
        'distance': 460,
        'formattedAddress': ['676 Kingston Rd. (at Main St.)',
         'Toronto ON M4E 1R4',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

In [68]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [69]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

NameError: name 'json_normalize' is not defined

In [70]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [74]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 